In [9]:
import torch
from torch import nn, optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

from utils.load_dataset import load_dataset

In [16]:
class NeuralNetwork(nn.Module):
    def __init__(self, n_input_features):
        super(NeuralNetwork, self).__init__()
        self.fully_connected_layer_1 = nn.Linear(n_input_features, 32)
        self.fully_connected_layer_2 = nn.Linear(32, 128)
        self.fully_connected_layer_3 = nn.Linear(128, 256)
        self.fully_connected_layer_4 = nn.Linear(256, 128)
        self.fully_connected_layer_5 = nn.Linear(128, 64)
        self.output_layer = nn.Linear(64, 2)
        
    def forward(self, x):
        x = torch.relu(self.fully_connected_layer_1(x))
        x = torch.relu(self.fully_connected_layer_2(x))
        x = torch.relu(self.fully_connected_layer_3(x))
        x = torch.relu(self.fully_connected_layer_4(x))
        x = torch.relu(self.fully_connected_layer_5(x))
        x = self.output_layer(x)
        return x

In [7]:
data = load_dataset()
assert len(data[0]) == 7

In [13]:
X = []
Y = []

for entry in data:
    features = entry[:6]
    targets = [entry[6][stop] for stop in sorted(entry[6])]
    X.append(features)
    Y.append(targets)

X = np.array(X, dtype=np.float32)
Y = np.array(Y, dtype=np.float32)


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Standardize features (optional but often recommended)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert arrays to PyTorch tensors
X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

# Create data loaders
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=64, shuffle=False)

In [17]:
model = NeuralNetwork(X_train.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
def train_model(num_epochs):
    model.train()
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluate the model
def evaluate_model():
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(f'Accuracy of the model on the test set: {100 * correct / total}%')


In [18]:
train_model(10)
evaluate_model()

RuntimeError: 0D or 1D target tensor expected, multi-target not supported